# Pipeline Database Exploration
Inspect and analyze the saved pipeline configuration data stored in `pipelines.db`.

In [ ]:
from pathlib import Path
import sqlite3

DB_PATH = Path("pipelines.db").resolve()
print(f"Using database at {DB_PATH}")

def get_connection():
    return sqlite3.connect(DB_PATH)


In [ ]:
try:
    import pandas as pd
    from IPython.display import display
    HAS_PANDAS = True
    print(f"pandas {pd.__version__} loaded")
except ImportError:
    pd = None
    display = None
    HAS_PANDAS = False
    print("pandas is not installed; results will be shown as plain records.")

def fetch_records(query, params=None, frame=None):
    params = params or ()
    with get_connection() as conn:
        conn.row_factory = sqlite3.Row
        cur = conn.execute(query, params)
        rows = cur.fetchall()
    records = [dict(row) for row in rows]
    if (frame or (frame is None and HAS_PANDAS)) and HAS_PANDAS:
        df = pd.DataFrame.from_records(records)
        if display:
            display(df)
        return df
    for record in records:
        print(record)
    return records


In [ ]:
schema_overview = fetch_records(
    """
    SELECT name,
           type
    FROM sqlite_master
    WHERE type IN ('table','index','trigger')
    ORDER BY type, name
    """
)
schema_overview


In [ ]:
# View pipelines table
pipelines_df = fetch_records(
    """
    SELECT id,
           name,
           description,
           data_source,
           technical_prompt_id,
           clinical_prompt_id,
           ensemble_enabled,
           ensemble_size,
           chain_of_verification,
           contextual_grounding,
           llm_as_judge,
           judge_model_id,
           created_at,
           updated_at
    FROM pipelines
    ORDER BY id
    """
)
pipelines_df


In [ ]:
# View pipeline steps with ordering
pipeline_steps_df = fetch_records(
    """
    SELECT p.id AS pipeline_id,
           p.name,
           pm.step_order,
           pm.model_id,
           pm.created_at
    FROM pipelines AS p
    LEFT JOIN pipeline_models AS pm
      ON pm.pipeline_id = p.id
    ORDER BY p.id, pm.step_order
    """
)
pipeline_steps_df


In [ ]:
if not HAS_PANDAS:
    print("Install pandas to unlock aggregated metrics.")
elif pipelines_df is None or pipelines_df.empty:
    print("No pipelines recorded yet.")
else:
    bool_cols = [
        "ensemble_enabled",
        "chain_of_verification",
        "contextual_grounding",
        "llm_as_judge",
    ]
    pipeline_metrics = pipelines_df.copy()
    pipeline_metrics[bool_cols] = pipeline_metrics[bool_cols].astype(bool)

    if isinstance(pipeline_steps_df, pd.DataFrame) and not pipeline_steps_df.empty:
        step_counts = (
            pipeline_steps_df[pipeline_steps_df["pipeline_id"].notna()]
            .groupby("pipeline_id")["model_id"]
            .count()
        )
    else:
        step_counts = pd.Series(dtype="int64")

    pipeline_metrics["model_steps"] = (
        pipeline_metrics["id"].map(step_counts).fillna(0).astype(int)
    )
    pipeline_metrics["mitigation_count"] = pipeline_metrics[bool_cols].sum(axis=1)

    summary_stats = pd.DataFrame(
        {
            "metric": [
                "Pipelines",
                "Average models per pipeline",
                "Median models per pipeline",
                "Pipelines with mitigations",
            ],
            "value": [
                len(pipeline_metrics),
                pipeline_metrics["model_steps"].mean().round(2),
                pipeline_metrics["model_steps"].median(),
                int((pipeline_metrics["mitigation_count"] > 0).sum()),
            ],
        }
    )
    display(summary_stats)

    strategy_totals = pipeline_metrics[bool_cols].sum().rename("pipelines_enabled").to_frame()
    strategy_totals.index.name = "strategy"
    display(strategy_totals)

    combinations = (
        pipeline_metrics.groupby(bool_cols, dropna=False)["id"]
        .count()
        .rename("pipeline_count")
        .reset_index()
        .sort_values("pipeline_count", ascending=False)
    )
    display(combinations)

    display(
        pipeline_metrics[
            [
                "id",
                "name",
                "model_steps",
                "mitigation_count",
                *bool_cols,
                "created_at",
                "updated_at",
            ]
        ].sort_values("updated_at", ascending=False)
    )

    pipeline_metrics
